In [1]:
import pandas as pd
import operator
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,make_scorer,classification_report,accuracy_score
from sklearn import svm
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve,roc_auc_score
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zairi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
jobs="./input/dataset/Modified.csv"
df=pd.read_csv(jobs)
df.head()

,RequiredQual,JobDescription,JobRequirment,Title,Combined
0,- Fluency in spoken and written English; knowl...,This is an excellent opportunity for the right...,- Responsible for managing office administrati...,Administrative Assistant,- Fluency in spoken and written English; knowl...
1,"- Written and oral fluency in English, Armenia...",The accountant is a full-time national staff p...,- Manage program financial system and budget o...,Accountant,"- Written and oral fluency in English, Armenia..."
2,- Excellent communication skills. Friendly per...,This is a full-time position based in CCDI's Y...,- Provide translation and interpretation servi...,Administrative Assistant,- Excellent communication skills. Friendly per...
3,"- Degree in Computer Science, Information Tech...","Synergy International Systems, Inc./Armenia se...",Specific tasks and key responsibilities includ...,Software Developer,"- Degree in Computer Science, Information Tech..."
4,- Advanced university degree in public health ...,The UNDP and the Ministry of Health seek\r\npr...,- Daily management of project activities by le...,Project Manager,- Advanced university degree in public health ...


In [3]:
df.shape
df.describe()

,RequiredQual,JobDescription,JobRequirment,Title,Combined
count,1790,1773,1773,1774,1774
unique,1578,1448,1475,20,1617
top,- Excellent communication skills;\r\n- Good or...,The incumbent will work under the direct super...,"- Answer telephone calls and inquiries, forwar...",Accountant,- Excellent communication skills; - Good oral ...
freq,43,55,50,216,40


In [4]:
classes = df['Title'].value_counts()[:20]
keys = classes.keys().to_list()

df = df[df['Title'].isin(keys)]
df['Title'].value_counts()

Accountant                                216
Chief Accountant                          151
Medical Representative                    151
Java Developer                            132
Sales Manager                             126
Administrative Assistant                  124
Lawyer                                    115
Project Manager                            94
Software Developer                         79
Web Developer                              74
Receptionist/ Administrative Assistant     73
Office Manager                             65
Marketing Specialist                       60
Marketing Manager                          59
QA Engineer                                59
PHP Developer                              52
Software Engineer                          51
Executive Assistant                        46
Brand Manager                              46
Python Developer                            1
Name: Title, dtype: int64

In [5]:
def chane_titles(x):
    x = x.strip()
    if x == 'Senior Java Developer':
        return 'Java Developer'
    elif x == 'Senior Software Engineer':
        return 'Software Engineer'
    elif x == 'Senior QA Engineer':
        return 'Software QA Engineer'
    elif x == 'Senior Software Developer':
        return 'Senior Web Developer'
    elif x =='Senior PHP Developer':
        return 'PHP Developer'
    elif x == 'Senior .NET Developer':
        return '.NET Developer'
    elif x == 'Senior Web Developer':
        return 'Web Developer'
    elif x == 'Database Administrator':
        return 'Database Admin/Dev'
    elif x == 'Database Developer':
        return 'Database Admin/Dev'

    else:
        return x
        
    
df['Title'] = df['Title'].apply(chane_titles)
df['Title'].value_counts()

Accountant                                216
Chief Accountant                          151
Medical Representative                    151
Java Developer                            132
Sales Manager                             126
Administrative Assistant                  124
Lawyer                                    115
Project Manager                            94
Software Developer                         79
Web Developer                              74
Receptionist/ Administrative Assistant     73
Office Manager                             65
Marketing Specialist                       60
Marketing Manager                          59
QA Engineer                                59
PHP Developer                              52
Software Engineer                          51
Executive Assistant                        46
Brand Manager                              46
Python Developer                            1
Name: Title, dtype: int64

In [6]:
qty=df['Title'].value_counts()[:5].tolist()
label=df['Title'].value_counts().index.tolist()
print(qty)

print('top 5 popular jobs : '+str(df['Title'].value_counts()[:5].index.tolist()))

[216, 151, 151, 132, 126]
top 5 popular jobs : ['Accountant', 'Chief Accountant', 'Medical Representative', 'Java Developer', 'Sales Manager']


In [7]:
stopwordsSkills=[]
sk= pd.read_csv("./input/dataset/skill2vec_50K.csv")['skills']
for word in sk:
    word=str(word)
    word.lower()
    word=word.split(',')
    if(word[0]!=''):
        stopwordsSkills.append(word[0])
sdf=pd.DataFrame({'skills':stopwordsSkills})


In [8]:
qtySkills=sdf["skills"].value_counts().tolist()
labelSkills=sdf['skills'].value_counts().index.tolist()
print('top 5 skills : \n'+str(sdf['skills'].value_counts()[:10]))

top 5 skills : 
Communication Skills    328
Java                    261
Sales                   205
Marketing               186
tools                   176
Business Development    164
Accounting              164
XML                     146
Finance                 145
development             144
Name: skills, dtype: int64


In [9]:
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(df['Title'].values)
analyze=vectorizer.build_analyzer()
features=vectorizer.get_feature_names()


In [10]:
jobSkills=[]
for i in df["RequiredQual"]:
    jobSkills.append(i.lower())
Xclass=vectorizer.fit_transform(jobSkills)
X_train,X_test,Y_train,Y_test=train_test_split(Xclass,df['Title'],test_size=0.2,random_state=42)


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

svm=SVC(C=5,gamma=1,kernel='rbf',probability=True)
svmfit=svm.fit(X_train,Y_train)
svm_predictions=svmfit.predict(X_test)
print(classification_report(Y_test,svm_predictions))

                                        precision    recall  f1-score   support

                            Accountant       0.77      0.83      0.80        41
              Administrative Assistant       0.49      0.93      0.64        27
                         Brand Manager       1.00      0.62      0.77         8
                      Chief Accountant       0.93      0.74      0.82        38
                   Executive Assistant       1.00      0.36      0.53        11
                        Java Developer       0.86      0.93      0.89        27
                                Lawyer       0.97      0.97      0.97        29
                     Marketing Manager       0.71      0.71      0.71         7
                  Marketing Specialist       0.90      0.60      0.72        15
                Medical Representative       0.97      1.00      0.98        30
                        Office Manager       0.80      0.36      0.50        11
                         PHP Developer 

In [37]:
userInput=input("skills :")
pred=vectorizer.transform([userInput.lower()])
output=svm.predict(pred)
print("you may look into " + output)

skills :law
['you may look into Lawyer']
